In [1]:
import pandas as pd
from ipycytoscape import *

df = pd.read_csv('data/argum_data_tmp.csv')
df['label'] = df['Item']

#df = pd.read_csv("https://raw.githubusercontent.com/nextstrain/ncov/fe5a7ed1f92af63d6d1d43d0307e4b2620108aaa/data/metadata.tsv", sep = '\t')

cytoscapeobj = CytoscapeWidget()
cytoscapeobj.set_tooltip_source('name')

cytoscapeobj.graph.add_graph_from_df(
    df=df[:30],
    groupby_cols=['Context'],
    attribute_list=['Item'],
    edges=tuple('Item'),
    directed=True
)

cytoscapeobj.graph.edges

[]

In [2]:
df = pd.DataFrame({
    'A': [2, 3, 3, 5],
    'B': [5,6,7,8]
})
df.loc[df.A % 3 == 0, 'B'] = 9
df
s = pd.Series([1, 2, 3], index=[4, 5, 6])
t = pd.Series([1, 2, 3], index=[5, 6, 7])
s + t

4    NaN
5    3.0
6    5.0
7    NaN
dtype: float64

In [3]:
import ipycytoscape

import json
#with open("data/concentricData.json") as fi:
#    json_file = json.load(fi)
with open("data/DAGData.json") as fi:
    json_file = json.load(fi)

cytoscapeobj = ipycytoscape.CytoscapeWidget()
cytoscapeobj.graph.add_graph_from_json(json_file)

cytoscapeobj.set_layout(name='dagre', nodeSpacing=10, edgeLengthVal=10)
cytoscapeobj.set_style([{
                        'selector': 'node',
                        'css': {
                            'background-color': '#11479e'
                            }
                        },
                        {
                        'selector': 'node:parent',
                        'css': {
                            'background-opacity': 0.333
                            }
                        },
                        {
                            'selector': 'edge',
                            'style': {
                                'width': 4,
                                'line-color': '#9dbaea',
                                'target-arrow-shape': 'triangle',
                                'target-arrow-color': '#9dbaea',
                                'curve-style': 'bezier'
                            }
                        }])

cytoscapeobj.graph.nodes
json_file

{'nodes': [{'data': {'id': 'n0'}},
  {'data': {'id': 'n1'}},
  {'data': {'id': 'n2'}},
  {'data': {'id': 'n3'}},
  {'data': {'id': 'n4'}},
  {'data': {'id': 'n5'}},
  {'data': {'id': 'n6'}},
  {'data': {'id': 'n7'}},
  {'data': {'id': 'n8'}},
  {'data': {'id': 'n9'}},
  {'data': {'id': 'n10'}},
  {'data': {'id': 'n11'}},
  {'data': {'id': 'n12'}},
  {'data': {'id': 'n13'}},
  {'data': {'id': 'n14'}},
  {'data': {'id': 'n15'}},
  {'data': {'id': 'n16'}}],
 'edges': [{'data': {'source': 'n0', 'target': 'n1'}},
  {'data': {'source': 'n1', 'target': 'n2'}},
  {'data': {'source': 'n1', 'target': 'n3'}},
  {'data': {'source': 'n4', 'target': 'n5'}},
  {'data': {'source': 'n4', 'target': 'n6'}},
  {'data': {'source': 'n6', 'target': 'n7'}},
  {'data': {'source': 'n6', 'target': 'n8'}},
  {'data': {'source': 'n8', 'target': 'n9'}},
  {'data': {'source': 'n8', 'target': 'n10'}},
  {'data': {'source': 'n11', 'target': 'n12'}},
  {'data': {'source': 'n12', 'target': 'n13'}},
  {'data': {'source':

In [4]:
from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD

# Defining the network structure
model = BayesianNetwork([("C", "H"), ("P", "H")])

# Defining the CPDs:
cpd_c = TabularCPD("C", 3, [[0.33], [0.33], [0.33]])
cpd_p = TabularCPD("P", 3, [[0.33], [0.33], [0.33]])
cpd_h = TabularCPD(
    "H",
    3,
    [
        [0, 0, 0, 0, 0.5, 1, 0, 1, 0.5],
        [0.5, 0, 1, 0, 0, 0, 1, 0, 0.5],
        [0.5, 1, 0, 1, 0.5, 0, 0, 0, 0],
    ],
    evidence=["C", "P"],
    evidence_card=[3, 3],
)

# Associating the CPDs with the network structure.
model.add_cpds(cpd_c, cpd_p, cpd_h)

# Some other methods
model.get_cpds()

[<TabularCPD representing P(C:3) at 0x7ff0a3dfce20>,
 <TabularCPD representing P(P:3) at 0x7ff0a3dfcfd0>,
 <TabularCPD representing P(H:3 | C:3, P:3) at 0x7ff0a3dfc970>]

In [5]:
# check_model check for the model structure and the associated CPD and returns True if everything is correct otherwise throws an exception
model.check_model()



True

In [6]:
# Infering the posterior probability
from pgmpy.inference import VariableElimination

infer = VariableElimination(model)
posterior_p = infer.query(["P"], evidence={"C": 0, "H": 2})
print(posterior_p)

0it [00:00, ?it/s]

0it [00:00, ?it/s]

+------+----------+
| P    |   phi(P) |
+======+==========+
| P(0) |   0.3333 |
+------+----------+
| P(1) |   0.6667 |
+------+----------+
| P(2) |   0.0000 |
+------+----------+
